## Dynamic Economic Dispatch (DED) with Curtailment and Storage (without transmission)
_Power System Optimization (with pyomo)_

**Problem Formulation**:  

Objective Function:

$$\min  TotalCost = \sum_{g \in G_{conv}, t} VarCost_g * P_{g,t} + \sum _{t} CurtailmentPenalty * P_t^{C} $$

subject to:
$$\begin{align}
    \sum_{g \in G} P_{g,t} + P_t^{dch} = Demand_t + P_t^{ch}& \quad  \forall t \in T & \quad \text{power balance} \\
    P_{g}^{min} \leq P_{g,t} \leq P_{g}^{max} &  \quad \forall g \in G , t \in T & \quad \text{generator limits} \\
    SoC_t = SoC_{t-1} + (\eta P_t^{ch} - \frac{P_t^{dch}} {\eta})\Delta t & \quad \forall t \in T & \quad \text{battery stage of charge balance}\\
    SoC^{min} \leq SoC_t \leq SoC^{max} &  \quad \forall t \in T  & \quad \text{stage of charge limits}\\
    0 \leq P_t^{ch} \leq P^{ch, max} &  \quad \forall t \in T  & \quad \text{charging limits}\\
    0 \leq P_t^{dch} \leq P^{dch, max} &  \quad \forall t \in T & \quad \text{discharging limits}\\
    P_{g,t+1} - P_{g,t} \leq RampUp_{g} & \quad \forall g \in G , t = 1..T-1 & \quad \text{ramp up limit}\\
    P_{g,t} - P_{g,t+1} \leq RampDn_{g} &  \quad \forall g \in G , t = 1..T-1 & \quad \text{ramp down limit}\\
    P_t^{C} =  \sum_{g \in G_{var-C}} (\Lambda_{g,t} - P_{g, t}) & \quad \forall t \in T  & \quad \text{curtailed power from curtailable variable generators}\\
    P_{g, t}^{var-NC} = \Lambda_{g,t}  & \quad \forall g \in G_{var-NC} , t \in T & \quad \text{power from non-curtailable variable generators}
\end{align}$$


 
$$P_{g,t}: \text{generation in MW, produced by each generator g at time t} \\
P_t^{C}: \text{sum of curtailed renewable power at time t}$$

Parameters:

- $P^{min}_g$, the minimum operating bounds for the generator (based on engineering or natural resource constraints)
- $P^{max}_g$, the maximum operating bounds for the generator (based on engineering or natural resource constraints)
- $Demand_t$ at time t in MW
- $SOC_t$: State of Charge at time t in MW
- $\eta$: battery efficiency
- $CurtPenalty$: curtailment panalty in  €/ MWh
- $\Lambda_{g,t}$: available capacity for generator g at time t
- Variable Costs:

$$
VarCost_g = VarOM_g + HeatRate_g \times FuelCost_g
$$
with OM: operating and maintenance.  
For renewables (variable generators) it reduces to $VarCost_g = VarOM_g$

### 1. Get the necessary data from Energy Charts and local files:

In [ ]:
import requests
import pandas as pd

url = "https://api.energy-charts.info/total_power"

end = pd.Timestamp.today() -pd.Timedelta(days=1)
start = end - pd.Timedelta(days=2)

params = {
    "country": "de" ,
    "start": round(start.timestamp()), 
    "end": round(end.timestamp()),
}

response = requests.get(url=url, params=params)

response.text

In [58]:
data = response.json()
index = data.pop(list(data.keys())[0])  # rely on dict order

df_dyn_generation_energy_charts = pd.DataFrame(
    {item['name']:item['data'] for item in data['production_types']},
    index=pd.DatetimeIndex(pd.to_datetime(index, unit='s'))
)

GW_to_MW = 1000
installed_capacity = pd.read_csv('data/energy-charts_Installierte_Netto-Leistung_de_2024.csv', index_col='Jahr').T.astype({2024:'float'})
installed_capacity.drop(installed_capacity.columns[0],axis=1, inplace=True)
installed_capacity = installed_capacity *  GW_to_MW 

installed_total_capacity = installed_capacity[2024].sum()

In [ ]:
import pandas as pd
df_generators=pd.read_csv('data/generators.csv', sep=',')
df_generators.fillna(0, inplace=True)

df_variables = df_generators.loc[df_generators.is_variable==1]
df_non_variables = df_generators.loc[df_generators.is_variable!=1]

df_generators

df_storage = pd.read_csv('data/storage.csv', index_col='Resource')
df_storage

### 2. Setting-up the pyomo model

In [60]:
curtailment_penalty = 100 # Euro/ MW

# battery parameters
battery_Emax = df_storage.loc['Battery', 'Emax'] #MWh
battery_initial_SoC = 0.5 * battery_Emax    #MWh
battery_charge_efficiency = 0.95
battery_discharge_efficiency = 0.9
battery_max_charge = 100 #MW
battery_max_discharge = 100 #MW

In [ ]:
df_variables[df_variables.Resource=="Solar"].index

In [ ]:
import pyomo.environ as pyo

n_generators = len(df_generators.index)
df_dyn_total_demand = df_dyn_generation_energy_charts["Load (incl. self-consumption)"] / installed_total_capacity * df_generators.Pmax.sum() /2  # simulate only partial network demand


model = pyo.ConcreteModel()

# Sets
model.G = pyo.RangeSet(n_generators)
model.G_var = pyo.Set(initialize=df_variables.index+1)
model.G_non_var = pyo.Set(initialize=df_non_variables.index+1)
model.G_var_NC = pyo.Set(initialize=df_variables[df_variables.Resource=="Solar"].index+1)
model.G_var_C = model.G_var - model.G_var_NC
model.T = pyo.RangeSet(len(df_dyn_generation_energy_charts))


#  Variables
def bounds_rule(model, g, t):
    return (df_generators.Pmin[g-1], df_generators.Pmax[g-1])

model.x = pyo.Var(
    model.G, 
    model.T,
    within=pyo.NonNegativeReals, 
    bounds=bounds_rule,
)

model.SoC = pyo.Var(model.T, within=pyo.NonNegativeReals, initialize=battery_initial_SoC, bounds=(0, battery_Emax))
model.Battery_charge = pyo.Var(model.T, within=pyo.NonNegativeReals, initialize=0, bounds=(0, battery_max_charge))
model.Battery_discharge = pyo.Var(model.T, within=pyo.NonNegativeReals, initialize=0, bounds=(0, battery_max_discharge))
model.u = pyo.Var(model.T, within=pyo.Binary) # charge / discharge switch
model.PC = pyo.Var(model.T, within=pyo.NonNegativeReals, initialize=0, bounds=(0, df_dyn_generation_energy_charts["Load (incl. self-consumption)"].max()))


# Params 

init_CF = {(g, t): 
    df_dyn_generation_energy_charts[df_generators.loc[g-1].Resource].iloc[t-1]/ installed_capacity.loc[df_generators.loc[g-1].Resource, 2024]
    for g in model.G_var 
    for t in model.T
} # simulate time dependent cf factors with real generation/ real installed capacity

model.VarOM = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.VarOM)))
model.HeatRate = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.HeatRate)))
model.FuelCost = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.FuelCost)))
model.CF = pyo.Param(model.G, model.T, initialize=init_CF)
model.Pmax = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.Pmax)))
model.Demand = pyo.Param(model.T, initialize=dict(zip(model.T, df_dyn_total_demand)))

model.ramp_up_percentage = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.RampUp)))
model.ramp_dn_percentage = pyo.Param(model.G, initialize=dict(zip(model.G, df_generators.RampDn)))

model.battery_initial_SoC=pyo.Param(initialize=battery_initial_SoC)
model.battery_charge_efficiency = pyo.Param(within=pyo.NonNegativeReals, initialize=battery_charge_efficiency)
model.battery_discharge_efficiency = pyo.Param(within=pyo.NonNegativeReals, initialize=battery_discharge_efficiency)
model.battery_max_charge = pyo.Param(within=pyo.NonNegativeReals, initialize=battery_max_charge)
model.battery_max_discharge = pyo.Param(within=pyo.NonNegativeReals, initialize=battery_max_discharge)


# Objective Function
model.total_costs=pyo.Objective(
    expr=sum(
            model.x[g,t] * (model.VarOM[g] + model.HeatRate[g] * model.FuelCost[g]) 
            for g in model.G 
            for t in model.T
        ) + sum(
            model.PC[t] * curtailment_penalty
            for t in model.T
        ),
    sense=pyo.minimize
)

# Constraints
def demand_rule(model, t):
    return sum(model.x[g,t] for g in model.G) + model.Battery_discharge[t] == model.Demand[t] + model.Battery_charge[t]
model.demand_constraint = pyo.Constraint(model.T, rule=demand_rule)

# capacity_constraint_non_var not necessary due to bounds rule

def capacity_rule_var_NC(model, g, t):
    return model.x[g, t] == model.CF[g, t] * model.Pmax[g]
model.capacity_constraint_var_NC = pyo.Constraint(model.G_var_NC, model.T, rule=capacity_rule_var_NC)

def capacity_rule_var_C(model, g, t):
    return model.x[g, t] <= model.CF[g, t] * model.Pmax[g]
model.capacity_constraint_var_C = pyo.Constraint(model.G_var_C, model.T, rule=capacity_rule_var_C)


def ramp_up_rule(model, g, t):
    if t < len(model.T):
        return model.x[g,t+1] - model.x[g, t] <= model.Pmax[g] * model.ramp_up_percentage[g]
    else:
        return pyo.Constraint.Skip
model.ramp_up_constraint = pyo.Constraint(model.G, model.T, rule=ramp_up_rule)

def ramp_down_rule(model, g, t):
    if t < len(model.T):
        return model.x[g,t] - model.x[g, t+1] <= model.Pmax[g] * model.ramp_dn_percentage[g]
    else:
        return pyo.Constraint.Skip
model.ramp_dn_constraint=pyo.Constraint(model.G, model.T, rule=ramp_down_rule)

def soc_rule(model, t):
    # delta t is 1 hour
    if t == 1:
        return model.SoC[t] == model.battery_initial_SoC + model.battery_charge_efficiency * model.Battery_charge[t] - model.Battery_discharge[t] / model.battery_discharge_efficiency
    else:
        return model.SoC[t] == model.SoC[t-1] + model.battery_charge_efficiency * model.Battery_charge[t] - model.Battery_discharge[t] / model.battery_discharge_efficiency
model.SoC_dynamics = pyo.Constraint(model.T, rule=soc_rule)

#Binary Constraint model.u[t]: Prevent simultaneous charging and discharging
def charge_rule(model, t):
    return model.Battery_charge[t] <= model.battery_max_charge * model.u[t]
model.battery_charge_limit = pyo.Constraint(model.T, rule=charge_rule)

def discharge_rule(model, t):
    return model.Battery_discharge[t] <= model.battery_max_discharge  * (1 - model.u[t])
model.battery_discharge_limit = pyo.Constraint(model.T, rule=discharge_rule)

def curtailment_rule(model, t):
    return model.PC[t] == sum(model.CF[g, t] * model.Pmax[g] - model.x[g, t] for g in model.G_var_C)
model.curtailment = pyo.Constraint(model.T, rule=curtailment_rule)

optimizer = pyo.SolverFactory("appsi_highs")
result = optimizer.solve(model)

result.write()
print(f"total costs: {pyo.value(model.total_costs)}")
print(f"curtailed power: {sum(pyo.value(model.PC[t]) for t in model.T)}")
# model.x.display()


### 3. Visualize optimized power generation 

In [ ]:

import matplotlib.pyplot as plt 

data = model.x.extract_values()
index = df_dyn_generation_energy_charts.index

df_data = pd.Series(data).unstack(level=0)
df_data.index = index
df_data.columns=[f"{df_generators.loc[generator-1].Resource}" for generator in df_data.columns]
df_data['Battery_out'] = [abs(pyo.value(model.Battery_discharge[t])) for t in model.T]
df_data['Battery_in'] = [-1*abs(pyo.value(model.Battery_charge[t])) for t in model.T]


# Visualize stacked optimized power generation
#  

ax = df_data[['Battery_in', 'Battery_out', 'Biomass', 'Gas_Turbine', 'Wind offshore', 'Wind onshore', 'Solar']].plot(kind='area', stacked=True, figsize=(10, 6), alpha = 0.7)
df_dyn_total_demand.plot(ax=ax, label='Demand', linestyle='--',color='red', linewidth=2)
df_SOC=pd.DataFrame([pyo.value(model.SoC[t]) for t in model.T], index=index, columns=['SoC'])
df_SOC.plot(ax=ax, label='SoC',color='black')
df_PC = pd.DataFrame([pyo.value(model.PC[t]) for t in model.T], index=index, columns=['PC'])
df_PC.plot(ax=ax, label='curtailed power', linestyle='--', color='blue')

plt.title('Stacked Power Generation with Demand Over Time')
plt.xlabel('Time')
plt.ylabel('Power (MW)')
plt.grid(True)
plt.legend(title='Generators and Demand',loc='lower right')


In [ ]:

X=[t for t in model.T]
XL=[str(t) for t in model.T]

SOC=[pyo.value(model.SoC[t]) for t in model.T]
Charge=[pyo.value(model.Battery_charge[t]) for t in model.T]
discharge=[pyo.value(model.Battery_discharge[t]) for t in model.T]

plt.plot(X,SOC,label='SOC')
plt.plot(X,Charge,label='Charge')
plt.plot(X,discharge,label='DisCharge')

plt.grid( which='major', color='black', linestyle='-')
plt.grid( which='minor', color='grey', linestyle='--')
plt.minorticks_on()
plt.legend()
plt.show()

https://www.mdpi.com/1996-1073/17/5/1257

https://www.sciencedirect.com/science/article/pii/S266679242100055X

https://www.gams.com/44/psoptlib_ml/libhtml/psoptlib_DEDESSwind.html
